Pour le scrapping des données, j'ai décidé d'utiliser `selenium`. Même s'il est moins rapide pour l'extraction de données simples, il fourni beaucoup plus de fonctionnalités, notamment l'extraction de données sur les sites qui utilisent du Javascript pour rendre les données dynamiquement sur le DOM (Document Object Model).

**Note Additionnelle:**

En raison de la lenteur constatée sur le scraping des données, nottement pour le Troisième lien qui  nécessite une requette aux pages individuelles des animeaux afin de pouvoir collecter les détails, `bs4` a été utilisé pour collecter les données depuis la page `HTML` récupérée par `driver.get(url)`

In [40]:
# Installing selenium and bs4 in the notebook environment
# !pip install selenium
# !pip install bs4

In [ ]:
# Importing dependencies
from typing import List, Tuple
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import sqlite3

In [42]:
# Setting up the environment (selenium driver options and get request)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # The browser will run en detached mode (in the background)
driver = webdriver.Chrome(options=options)

Pour le premier et les deux derniers liens, nous pouvons directement récupérer les informations, nottament:

- le nom,
- le prix,
- l'adresse,
- le lien vers l'image de l'animal (même si ce dernier conduit vers le thumbnail, qui est de moindre qualité).

Par contre pour pour le troisième lien, nous devons impérativement envoyer une nouvelle requette HTTP à chacune des pages individuelles des animaux.
Cette requette additionnelle aura une force incidence sur les performances de l'application, qui est proportionnelle au nombre de page a scraper.

In [43]:
# Number of page to be scraped for each of the link
NUM_PAGE = 2

# Website Base URL, will be used to reconstruct image URLs
BASE_URL = "https://sn.coinafrique.com"

# Creating a list of urls to be scraped
ROOT_PAGE_LIST = [
    {
        "title": "chiens",
        "url": "https://sn.coinafrique.com/categorie/chiens?page=",
        "keys": ("nom", "prix", "adresse", "image_lien")
    },
    {
        "title": "moutons",
        "url": " https://sn.coinafrique.com/categorie/moutons?page=",
        "keys": ("nom", "prix", "adresse", "image_lien")
    },
    {
        "title": "poules_lapins_et_pigeons",
        "url": "https://sn.coinafrique.com/categorie/poules-lapins-et-pigeons?page=",
        "keys": ("details", "prix", "adresse", "image_lien"),
    },
    {
        "title": "autres_animaux",
        "url": "https://sn.coinafrique.com/categorie/autres-animaux?page=",
        "keys": ("nom", "prix", "adresse", "image_lien"),
    }
]


In [ ]:
def scrap_simple(container):
    v1 = container.find("p", class_="ad__card-description")
    v2 = container.find("p", class_="ad__card-price")
    v3 = container.find("p", class_="ad__card-location")
    v4 = container.find("img", class_="ad__card-img")

    return {
        "nom": v1.a.text if v1 is not None else np.nan,
        "prix": v2.a.text if v2 is not None else "",
        "adresse": v3.span.text if v3 is not None else np.nan,
        "image_lien": v4["src"] if v4 is not None else np.nan
    }


def scrap_nested(path, base_url=None):
    url = BASE_URL + path if base_url is None else base_url + path

    driver.get(url)
    time.sleep(1)
    soup = bs(driver.page_source, "html.parser")

    v1 = soup.find("div", class_="ad__info__box ad__info__box-descriptions")
    v2 = soup.find("p", class_="price")

    v3 = soup.find("div", class_="row valign-wrapper extra-info-ad-detail")
    v3 = v3 if (v3 is None) else v3.find_all("span")
    v3 = v3 if (v3 is None) else v3[2]
 
    v4 = soup.find(id="slider")
    v4 = v4 if (v4 is None) else v4.find_all("div")[0]

    return {
        "details": v1.find_all("p")[-1].text if (v1 is not None) else np.nan,
        "prix": v2.text if (v1 is not None) else "",
        "adresse": v3.text if (v3 is not None) else np.nan,
        "image_lien": v4["style"].split('(')[1].split(')')[0].strip('"') if (v4 is not None) else np.nan,
    }

Dans la cellule suivante, une liste de tuple, contenant le nom de la page et une référence vers le DataFrame en mémoire a été créée. Le but est de s'en servir pour exporter les données (DataFrame vers une base de donnée sqlit3) dans une seule boucle, après avoir néttoyer les données.

In [ ]:
dfs: List[Tuple[str, pd.DataFrame]] = []  # [("DataFrame Name", "Reference to DataFrame")]

# Looping over ROOT_PAGE_LIST to scrap data from all URLs

for page in ROOT_PAGE_LIST:
    animal_data_list = []

    for page_index in range(1, NUM_PAGE + 1):
        page_url = f"{page["url"]}{page_index}"
        driver.get(page_url)
        time.sleep(1)

        soup = bs(driver.page_source, 'html.parser')
        containers = soup.find_all("div", class_="col s6 m4 l3")

        if page['title'] != "poules_lapins_et_pigeons":
            animal_data_list.extend([scrap_simple(container) for container in containers])
        else:
            path_list = [container.find("p", class_="ad__card-description").a["href"] for container in containers]

            animal_data_list.extend([scrap_nested(path) for path in path_list])

    dfs.append((page["title"], pd.DataFrame(animal_data_list)))

Les informations que nous avons collecter sur la platforme, coinAfrique sont requisent par la modération pour qu'une annonce soit validée.
Ceci dit, nous allons quand même les imputer pour éviter tout désagrément, au cas où le script manque à collecter une ou plusieurs données.

Les prix, peuvent être `Sur Demande`, une chaîne de caractère. En convertissant cette colonne en numérique, ces valeurs seront transformées en NaN.

Vu que ces valeurs manquantes pour `le prix` sont complètement aléatoires, nous pouvons les imputer:
- par la moyenne si la distribution est symétrique et sans valeurs aberrantes
- par la médiane si la distribution est asymétrique ou avec des valeurs aberrantes

In [ ]:
# plt.boxplot(dfs[0][1]["prix"], orientation="vertical")
# plt.show()

: 

In [ ]:
# plt.hist(dfs[0][1]["prix"])
# plt.show()

: 

La méthode d'imputation approprié pour le prix, après visualisation est:
- Une imputation simple par la mediane.

Vu que les distributions sont asymétriques et avec des valeurs aberrantes.

Pour les colonnes qualitatives:
- `name`: la modalité la plus fréquente `mode`
- `details`: une constante `Details Inconnu`
- `adresse`: une constante `Adresse Inconnu`
- `image_lien`: une constante `Lien Inconnu`

In [ ]:
for _, df in dfs:
    df = df.drop_duplicates()
    # prices with space in them, results to NaN, so we remove the spaces and the currencies
    df["prix"] = pd.to_numeric(df["prix"].str.replace(" ", "").str.rstrip("CFA"), errors="coerce")
    # Replacing missing prices by the Median
    df["prix"] = df["prix"].fillna(df["prix"].median())

    # Replacing missing adress and image_link with a constant
    df['adresse'] = df['adresse'].fillna('Adresse Inconnue')
    df['image_lien'] = df['image_lien'].fillna('Lien Inconnu')

    # Replacing Name/Details with a constant
    if "nom" in df.columns:
        df["nom"] = df["nom"].fillna(df["nom"].mode())
    elif "details" in df.columns:
        df["details"] = df["details"].fillna("Details Inconnus")

In [51]:
# Exporting the data to a sqlite database

# Create a connexion with a sqlite database (Animals.db)
conn = sqlite3.connect('Animals.db')

# Create a cursor (enabling interactions with the database)
c = conn.cursor()

# Create a table for each DataFrame and export them
for title, df in dfs:
    df.to_sql(name=title, con=conn, if_exists="replace", index=False)
